In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPRegressor
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import csv
import time
import math
from sklearn import preprocessing, cross_validation, neighbors
import compute_measure
import pandas as pd
from numpy import genfromtxt, savetxt

In [2]:

df = pd.read_csv("data.csv")

X = np.array(df.drop(['class'],1))
y = np.array(df['class'])

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)
#split 10% of the dataset to test the performance of each classifiers
A_train, A_test, b_train, b_test = cross_validation.train_test_split(X_train,y_train,test_size=0.2)

names = [ "k-NN",
		 "DecisionTree",
		 "RandomForest",
		 "AdaBoost",
		 "BaggingRegressor",
		 "Gradient Boosting",
		 "MLP",
		 "Bayesian Ridge"]


regressors = [
	KNeighborsRegressor(3),
	DecisionTreeRegressor(max_depth=5),
	RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1),
	AdaBoostRegressor(),
	BaggingRegressor(),
	GradientBoostingRegressor(),
	MLPRegressor(alpha=1e-5, hidden_layer_sizes=(100, 1),  max_iter=5000),
	linear_model.BayesianRidge()]


def get_MSE(Error):
	mse=np.sum(np.power(Error,2))
	mse=mse/len(Error)
	return mse

In [ ]:
copy_names = list(names)
MSE = []
for name, reg in zip(names, regressors):
	reg.fit(A_train, b_train)
	Error_reg   = [None] * len(A_train)
	predictedIV = reg.predict(A_test)
	Error_reg   = abs(b_test - predictedIV)
	print('\n\nThe Model '+ name + ' Peformance Summary as follows:')
	print('The MSE is           {:20.16f}'.format(get_MSE(Error_reg)))
	print('The mean error is    {:20.16f}'.format(np.mean(Error_reg)))
	print('The maximum error is {:20.16f}'.format(max(Error_reg)))
	print('The minimum error is {:20.16f}'.format(min(Error_reg)))
	MSE.append(get_MSE(Error_reg))

new_regressors = sorted(zip(MSE, copy_names, regressors))[:3]
##store value.

print("\n\nTop Three Regressors:")

for MSE, name, reg in new_regressors:
	print(name)
	print('The MSE is           {:20.16f}'.format(MSE))




The Model k-NN Peformance Summary as follows:
The MSE is             0.0154589371980676
The mean error is      0.0131400966183575
The maximum error is   2.0000000000000000
The minimum error is   0.0000000000000000


The Model DecisionTree Peformance Summary as follows:
The MSE is             0.0157379503979114
The mean error is      0.0136463696409027
The maximum error is   2.0000000000000000
The minimum error is   0.0000000000000000


The Model RandomForest Peformance Summary as follows:
The MSE is             0.0976665257445019
The mean error is      0.2134431345488068
The maximum error is   1.7150975294547424
The minimum error is   0.0063882020280722


The Model AdaBoost Peformance Summary as follows:
The MSE is             0.0400302573381748
The mean error is      0.1754870189507745
The maximum error is   1.7070175438596491
The minimum error is   0.0105777054515866


The Model BaggingRegressor Peformance Summary as follows:
The MSE is             0.0047845410628019
The mean error

In [4]:

print("\n\n\n")
print("Consensus Learning Method:")


prediction_results = []

new_MSE = []
new_name_list = []
new_regressors_list = []
for MSE, name, reg in new_regressors:
	reg.fit(X_train, y_train)
	Error_reg   = [None] * len(X_train)
	predictedIV = reg.predict(X_test)
	prediction_results.append(predictedIV)
	Error_reg   = abs(y_test - predictedIV)
	print('\n\nThe Model '+ name + ' Peformance Summary as follows:')
	print('The MSE is           {:20.16f}'.format(get_MSE(Error_reg)))
	print('The mean error is    {:20.16f}'.format(np.mean(Error_reg)))
	print('The maximum error is {:20.16f}'.format(max(Error_reg)))
	print('The minimum error is {:20.16f}'.format(min(Error_reg)))
	new_MSE.append(get_MSE(Error_reg))
	new_name_list.append(name)

sorted_regressors = sorted(zip(new_MSE, new_name_list, prediction_results))
print('\n\nSorted:')
sorted_new_MSE = []
for MSE, name, pred in sorted_regressors:
	print(name)
	print(MSE)
	sorted_new_MSE.append(MSE)








Consensus Learning Method:


The Model BaggingRegressor Peformance Summary as follows:
The MSE is             0.0038274849281187
The mean error is      0.0036790848662854
The maximum error is   2.0000000000000000
The minimum error is   0.0000000000000000


The Model Gradient Boosting Peformance Summary as follows:
The MSE is             0.0081577757001216
The mean error is      0.0204459644548420
The maximum error is   1.2182904085987689
The minimum error is   0.0000252301205889


The Model k-NN Peformance Summary as follows:
The MSE is             0.0154583397743082
The mean error is      0.0124697274179420
The maximum error is   2.0000000000000000
The minimum error is   0.0000000000000000


Sorted:
BaggingRegressor
0.00382748492812
Gradient Boosting
0.00815777570012
k-NN
0.0154583397743


In [5]:
# weighting algorithm #1
weights_1 = (1/sorted_new_MSE[0])/(1/np.sum(sorted_new_MSE))
weights_2 = (1/sorted_new_MSE[1])/(1/np.sum(sorted_new_MSE))
weights_3 = (1/sorted_new_MSE[2])/(1/np.sum(sorted_new_MSE))

# weighting algorithm #2
weights_4 = (1/math.pow(sorted_new_MSE[0], 2))/(1/math.pow(np.sum(sorted_new_MSE), 2))
weights_5 = (1/math.pow(sorted_new_MSE[1], 2))/(1/math.pow(np.sum(sorted_new_MSE), 2))
weights_6 = (1/math.pow(sorted_new_MSE[2], 2))/(1/math.pow(np.sum(sorted_new_MSE), 2))

# weighting algorithm #2
weights_7 = (1/math.log(math.pow(sorted_new_MSE[0], 2)))/(1/math.log(math.pow(np.sum(sorted_new_MSE), 2)))
weights_8 = (1/math.log(math.pow(sorted_new_MSE[1], 2)))/(1/math.log(math.pow(np.sum(sorted_new_MSE), 2)))
weights_9 = (1/math.log(math.pow(sorted_new_MSE[2], 2)))/(1/math.log(math.pow(np.sum(sorted_new_MSE), 2)))



In [7]:
print(weights_1)

7.17013937819


In [6]:
print("\n\n\n")
merged_1 = np.zeros_like(prediction_results[0])
merged_1.fill(0)

for i in range(merged_1.size):
	prediction_sum = prediction_results[0][i]*weights_1 + prediction_results[1][i]*weights_2 + prediction_results[2][i]*weights_3
	print(prediction_sum)
	if prediction_sum >= 0.9:
		merged_1[i] = 1
	if prediction_sum <= -0.9:
		merged_1[i] = -1
	if prediction_sum < 0.9 and prediction_sum > -0.9:
		merged_1[i] = 0





-12.3119852238
-12.2956954231
-12.3318955491
-12.309484342
-12.3083868136
-12.3083868136
12.3214729662
-12.3042118217
-12.3482676684
-12.3083868136
-12.2979432038
9.13237464415
-12.3042118217
-12.3042118217
-12.3053548349
-12.2979432038
12.3434902312
-12.1823200626
-12.3042118217
-12.3042118217
-12.3013047357
-12.3595822327
-12.3083868136
-12.3013047357
-12.3083868136
12.2030916091
-12.3042118217
-12.3121455901
-12.306076264
-12.3083868136
-12.3011798431
12.0819055485
-12.3083868136
12.3297273405
-12.2979432038
-12.306076264
-12.3123914281
-12.3083868136
-12.3083868136
-12.3521730975
-12.2938309809
-12.3083868136
-12.3042118217
-12.2979432038
-12.3134109404
-12.3042118217
12.1487697847
-12.3450351393
-12.306076264
-12.3042118217
-11.2245554776
-12.0930673078
12.2915751741
-12.3083868136
-12.3083868136
-12.3119852238
-12.3054797275
-12.3042118217
-12.3054797275
-12.2979432038
-12.3093608412
-12.3546226297
-12.3013047357
-12.2863316477
-12.2984940908
12.2822514985
-12.3042118217
-12.

-12.3083868136
-12.3059993094
-12.3013047357
-12.6090477316
-12.3083868136
12.3140894026
-11.2418936314
-12.2979432038
-12.3083868136
-12.3042118217
-12.2950361178
-12.309484342
12.2887808063
12.2962192641
-12.3053548349
-12.3013047357
12.2929958605
-12.3042118217
12.2043802523
-12.5502798076
-12.3042118217
12.521421419
-12.2979432038
-12.306076264
-12.3042118217
-12.3625873307
-12.3042118217
-12.3164429191
12.2822514985
-12.303169178
-12.3083868136
-12.3042118217
-11.604690895
-12.3013047357
-12.3083868136
11.8871740261
-12.3042118217
12.2370692299
-12.3042118217
-12.5710376507
-12.3083868136
12.2835649113
-12.2950361178
-12.3013047357
-12.3013047357
-12.3039291183
-12.3013047357
-12.3121455901
-12.3013047357
-12.3427245897
-12.3042118217
-12.3442418846
-12.3083868136
-12.3083868136
-12.0609728191
-12.0586835001
12.2526868892
-12.3053548349
-12.3042118217
-12.3083868136
-12.3013047357
-12.3013047357
-12.3083868136
-12.3011798431
12.3047561735
12.2917000667
2.68229043677
11.8969018664


-12.2979432038
-12.3131282369
-12.3083868136
-12.3013047357
-12.2979432038
-11.8938878308
12.3009028328
-12.2979432038
-12.3117284169
-12.3083868136
-12.3027781924
12.325309383
-12.3083868136
-12.3042118217
12.2778208724
-12.2950988867
12.2849796618
11.8988563582
-12.3083868136
12.2787641183
-12.276102761
11.4055863508
-12.306076264
-12.2950988867
12.3045149243
-12.306076264
-12.3053548349
-12.2957950311
-12.394384449
-12.3083868136
-12.3437672335
12.2884371845
-10.6938671006
12.2884371845
-12.2979432038
12.2854314488
-12.3042118217
-12.2938309809
-12.3408601475
-12.2950361178
-12.4588033189
-12.1240700069
-12.3042118217
-12.3042118217
-12.3054797275
-12.4065262157
-12.3083868136
-12.287562363
12.2445561258
-12.3083868136
-12.3042118217
-12.3042118217
12.3048172534
-12.3083868136
-12.3054797275
-12.3013047357
12.3017320824
-12.2979432038
11.878716006
-12.3083868136
-12.3013047357
-12.3054797275
-12.3083868136
-12.3042118217
11.0082150408
-12.3013047357
-12.303169178
-12.3013047357
-12.

-12.3042118217
-12.3042118217
-12.3083868136
-12.3083868136
-12.3013047357
-12.2815432488
-12.4713850192
-12.3086910874
12.7426418472
-12.3083868136
-12.3408601475
-12.3011798431
-12.3122679273
-12.4588033189
-12.3083868136
-12.3054797275
-12.2989800004
9.27034529601
-12.3083868136
-12.306076264
12.3215893227
-12.306076264
3.21426437043
12.3094203934
-12.3054797275
-12.3580855729
12.2871413907
-12.3083868136
-12.2979432038
-12.2979432038
-12.3083868136
12.2887198879
-12.3042118217
-12.3081041101
11.4763843148
12.2848989332
-12.3090781378
-12.3042118217
-12.3123914281
-12.309484342
-12.3427245897
-12.3093608412
12.2917000667
-12.2950361178
12.3274725574
-12.1097150998
-12.3042118217
12.2885620771
-12.2950361178
-12.3620089813
-12.2979432038
-12.3011798431
-12.3083868136
-12.3063766027
-12.3083868136
-12.2950361178
-12.3172242895
-12.3013047357
-12.306076264
-12.3083868136
-12.3098870755
-12.3083868136
-12.3042118217
-12.3011798431
12.3140894026
-12.2826957401
-12.3042118217
-12.39438444

-12.3042118217
-12.3042118217
-12.3042118217
-12.3626787352
-12.3054797275
-12.306076264
-12.2979432038
12.3271877401
12.2917000667
-12.3083868136
-12.3400668928
-12.3083868136
10.9285622918
-12.3027781924
-12.2979432038
-12.2979432038
-12.2950361178
-12.3013047357
-12.3121455901
-12.3042118217
-12.3042118217
-12.2979432038
-12.3083868136
-12.3939568182
-12.3083868136
-12.3042118217
-12.2938309809
12.3703238568
-12.2979432038
-12.3013047357
12.2807414709
-12.2986032006
12.3554955931
-12.2950361178
12.3182804589
-12.3054797275
-12.2950361178
12.3556204857
-12.306076264
-12.3042118217
-12.3013047357
-12.2979432038
12.2870100889
12.2884371845
-12.3083868136
-12.3083868136
-12.3083868136
-12.3083868136
-12.3083868136
-12.2979432038
-12.3042118217
12.0757668735
-12.3188451154
-12.3083868136
-12.3054797275
12.2871413907
11.9581423466
-12.4400330072
-12.3042118217
-11.2853612071
-12.2950361178
12.247915939
-12.3053548349
-11.3058720396
-12.2976605004
-12.3836446752
-12.3083868136
-12.30421182

In [7]:
merged_2 = np.zeros_like(prediction_results[0])
merged_2.fill(0)


for i in range(merged_2.size):
	prediction_sum = prediction_results[0][i]*weights_4 + prediction_results[1][i]*weights_5 + prediction_results[2][i]*weights_6
	if prediction_sum >= 0.9:
		merged_2[i] = 1
	if prediction_sum <= -0.9:
		merged_2[i] = -1
	if prediction_sum < 0.9 and prediction_sum > -0.9:
		merged_2[i] = 0

In [8]:
merged_3 = np.zeros_like(prediction_results[0])
merged_3.fill(0)


for i in range(merged_3.size):
	prediction_sum = prediction_results[0][i]*weights_7 + prediction_results[1][i]*weights_8 + prediction_results[2][i]*weights_9
	if prediction_sum >= 0.9:
		merged_3[i] = 1
	if prediction_sum <= -0.9:
		merged_3[i] = -1
	if prediction_sum < 0.9 and prediction_sum > -0.9:
		merged_3[i] = 0

In [9]:
counter_1 = 0
counter_positive_1 = 0
for i in range(merged_1.size):
    counter_1 = counter_1 + 1
    if merged_1[i] != 0:
        if merged_1[i] == y_test[i]:
            counter_positive_1 = counter_positive_1 + 1


In [10]:
counter_2 = 0
counter_positive_2 = 0
for i in range(merged_1.size):
    counter_2 = counter_2 + 1
    if merged_2[i] != 0:
        if merged_2[i] == y_test[i]:
            counter_positive_2 = counter_positive_2 + 1

In [11]:
counter_3 = 0
counter_positive_3 = 0
for i in range(merged_1.size):
    counter_3 = counter_3 + 1
    if merged_3[i] != 0:
        if merged_3[i] == y_test[i]:
            counter_positive_3 = counter_positive_3 + 1

In [12]:
print("\nweighting algorithm #1")
print("Total Test Size:")
print(y_test.size)
print("Data Predicted (prediction sum value > 0.9 or <-0.9):")
print(counter_1)
print("Data Predicted Correctly:")
print(counter_positive_1)
accuracy_1 = float(counter_positive_1)/counter_1
print("Accuracy:")
print(accuracy_1)
print("Misclassified Number:")
misclassified_number_1 = (1-accuracy_1)*counter_1
print(misclassified_number_1)


weighting algorithm #1
Total Test Size:
6469
Data Predicted (prediction sum value > 0.9 or <-0.9):
6469
Data Predicted Correctly:
6466
Accuracy:
0.999536249807
Misclassified Number:
3.0


In [13]:
print("\nweighting algorithm #2")
print("Total Test Size:")
print(y_test.size)
print("Data Predicted (prediction sum value > 0.9 or <-0.9):")
print(counter_2)
print("Data Predicted Correctly:")
print(counter_positive_2)
accuracy_2 = float(counter_positive_2)/counter_2
print("Accuracy:")
print(accuracy_2)
print("Misclassified Number:")
misclassified_number_2 = (1-accuracy_2)*counter_2
print(misclassified_number_2)


weighting algorithm #2
Total Test Size:
6469
Data Predicted (prediction sum value > 0.9 or <-0.9):
6469
Data Predicted Correctly:
6466
Accuracy:
0.999536249807
Misclassified Number:
3.0


In [14]:
print("\nweighting algorithm #3")
print("Total Test Size:")
print(y_test.size)
print("Data Predicted (prediction sum value > 0.9 or <-0.9):")
print(counter_3)
print("Data Predicted Correctly:")
print(counter_positive_3)
accuracy_3 = float(counter_positive_3)/counter_3
print("Accuracy:")
print(accuracy_3)
print("Misclassified Number:")
misclassified_number_3 = (1-accuracy_3)*counter_3
print(misclassified_number_3)


weighting algorithm #3
Total Test Size:
6469
Data Predicted (prediction sum value > 0.9 or <-0.9):
6469
Data Predicted Correctly:
6412
Accuracy:
0.991188746329
Misclassified Number:
57.0
